<a href="https://colab.research.google.com/github/AuFeld/DS-Unit-2-Kaggle-Challenge/blob/master/module3/Chase_Goldfeld_DS_223_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


You won't be able to just copy from the lesson notebook to this assignment.

- Because the lesson was ***regression***, but the assignment is ***classification.***
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [0]:
# pre-req 1: import libraries:

In [106]:
import numpy as np
import pandas as pd
import plotly.express as px
!pip install category_encoders
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
import graphviz
from sklearn.tree import export_graphviz
import seaborn as sns
import itertools
from math import floor
from ipywidgets import interact
from sklearn.tree import DecisionTreeRegressor, export_graphviz
from sklearn.ensemble import BaggingClassifier, ExtraTreesClassifier, RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import validation_curve
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
import pandas_profiling

In [0]:
# look at the top five most important features

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

# Split train into train and test
train, test = train_test_split(train, train_size=0.90, test_size=0.10, 
                              stratify=train['status_group'], random_state=42)

In [0]:
# pre-req 2: define a function to wrangle train, validate, and test sets in the same way. 
#            Clean outliers and engineer features.

In [0]:
def wrangle(X):
    """Wrangle train, validate, and test sets in the same way"""
    
    # Prevent SettingWithCopyWarning
    X = X.copy()
    
    # About 3% of the time, latitude has small values near zero,
    # outside Tanzania, so we'll treat these values like zero.
    X['latitude'] = X['latitude'].replace(-2e-08, 0)
    
    # When columns have zeros and shouldn't, they are like null values.
    # So we will replace the zeros with nulls, and impute missing values later.
    # Also create a "missing indicator" column, because the fact that
    # values are missing may be a predictive signal.
    cols_with_zeros = ['longitude', 'latitude', 'construction_year', 
                       'gps_height', 'population']
    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)
        X[col+'_MISSING'] = X[col].isnull()
            
    # Drop duplicate columns
    duplicates = ['quantity_group', 'payment_type']
    X = X.drop(columns=duplicates)
    
    # Drop recorded_by (never varies) and id (always varies, random)
    unusable_variance = ['recorded_by', 'id']
    X = X.drop(columns=unusable_variance)
    
    # Convert date_recorded to datetime
    X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)
    
    # Extract components from date_recorded, then drop the original column
    X['year_recorded'] = X['date_recorded'].dt.year
    X['month_recorded'] = X['date_recorded'].dt.month
    X['day_recorded'] = X['date_recorded'].dt.day
    X = X.drop(columns='date_recorded')
    
    # Engineer feature: how many years from construction_year to date_recorded
    X['years'] = X['year_recorded'] - X['construction_year']
    X['years_MISSING'] = X['years'].isnull()
    
    # return the wrangled dataframe
    return X

In [0]:
train = wrangle(train)
#val = wrangle(val)
test = wrangle(test)

In [113]:
train['status_group'].value_counts()

functional                 29033
non functional             20542
functional needs repair     3885
Name: status_group, dtype: int64

In [114]:
val['status_group'].value_counts()

functional                 8065
non functional             5706
functional needs repair    1079
Name: status_group, dtype: int64

In [115]:
status_group_breakdown = pd.crosstab(train['region_code'], train['status_group'], normalize='index')
status_group_breakdown

status_group,functional,functional needs repair,non functional
region_code,,,
1,0.449671,0.096611,0.453718
2,0.652110,0.057615,0.290275
3,0.603960,0.072607,0.323432
4,0.563910,0.028748,0.407342
5,0.534389,0.073690,0.391921
6,0.662759,0.010345,0.326897
7,0.574825,0.004196,0.420979
8,0.087273,0.050909,0.861818
9,0.363380,0.056338,0.580282


In [0]:
top10 = train['subvillage'].value_counts()[:10].index
 
# At locations where the neighborhood is NOT in the top 10,
# replace the neighborhood with 'OTHER'
train.loc[~train['subvillage'].isin(top10), 'subvillage'] = 'OTHER'
test.loc[~test['subvillage'].isin(top10), 'subvillage'] = 'OTHER'

In [117]:
# The status_group column is the target
target = 'status_group'

# Get a dataframe with all train columns except the target
train_features = train.drop(columns=[target])

# Get a list of the numeric features
numeric_features = train_features.select_dtypes(include='number').columns.tolist()

# Get a series with the cardinality of the nonnumeric features
cardinality = train_features.select_dtypes(exclude='number').nunique()

# Get a list of all categorical features with cardinality <= 50
categorical_features = cardinality[cardinality <= 50].index.tolist()

# Combine the lists 
features = numeric_features + categorical_features
print(features)

['amount_tsh', 'gps_height', 'longitude', 'latitude', 'num_private', 'region_code', 'district_code', 'population', 'construction_year', 'year_recorded', 'month_recorded', 'day_recorded', 'years', 'basin', 'subvillage', 'region', 'public_meeting', 'scheme_management', 'permit', 'extraction_type', 'extraction_type_group', 'extraction_type_class', 'management', 'management_group', 'payment', 'water_quality', 'quality_group', 'quantity', 'source', 'source_type', 'source_class', 'waterpoint_type', 'waterpoint_type_group', 'longitude_MISSING', 'latitude_MISSING', 'construction_year_MISSING', 'gps_height_MISSING', 'population_MISSING', 'years_MISSING']


In [0]:
# Arrange data into X features matrix and y target vector 
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

In [0]:
# adapt the example which is good real-world practice
# 1. Use a model for classification, such as RandomForestClassifier
# 2. USe hyperparamteres that match the classifier, such as randomforestclassifier_ ... 
# 3. Use a metric for classification, such as scoring ='accuracy' 
# 4. If you're doing a multi-class classification problem - 
#    such as whether a waterpump is functional, functional needs repair, or nonfunctional
#    then use a categorical encoding that works for multi-class classification, 
#    such as Ordinal Encoder

In [0]:
# objective 1: use sci-kit learn for hyperparameter optimization with RandomizedSearchCV

In [0]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

In [122]:
RandomForestClassifier.get_params(RandomForestClassifier, deep=True)

{'bootstrap': None,
 'class_weight': None,
 'criterion': None,
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': None,
 'min_impurity_split': None,
 'min_samples_leaf': None,
 'min_samples_split': None,
 'min_weight_fraction_leaf': None,
 'n_estimators': None,
 'n_jobs': None,
 'oob_score': None,
 'random_state': None,
 'verbose': None,
 'warm_start': None}

In [123]:
RandomizedSearchCV.get_params(RandomizedSearchCV, deep=True)

{'cv': None,
 'error_score': None,
 'estimator': None,
 'iid': None,
 'n_iter': None,
 'n_jobs': None,
 'param_distributions': None,
 'pre_dispatch': None,
 'random_state': None,
 'refit': None,
 'return_train_score': None,
 'scoring': None,
 'verbose': None}

In [124]:
pipeline = make_pipeline(
  ce.OrdinalEncoder(), 
  SimpleImputer(), 
  StandardScaler(),
  RandomForestClassifier(random_state=42)
)

param_distributions = {
    'simpleimputer__strategy': ['mean', 'median'], 
    'randomforestclassifier__n_estimators': randint(50, 150),
    'randomforestclassifier__criterion': ["gini", "entropy"],
}

search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=5, 
    cv=2, 
    scoring='accuracy', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)

# fit on train, score on val

search.fit(X_train, y_train);

Fitting 2 folds for each of 5 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   15.3s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   36.1s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.2min finished


In [125]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation MAE', -search.best_score_)

Best hyperparameters {'randomforestclassifier__criterion': 'entropy', 'randomforestclassifier__n_estimators': 140, 'simpleimputer__strategy': 'mean'}
Cross-validation MAE -0.794668911335578


In [0]:
# See detailed results

In [127]:
pd.DataFrame(search.cv_results_).sort_values(by='rank_test_score').T

,1,0,2,4,3
mean_fit_time,16.378,10.1871,11.2684,9.25388,7.55715
std_fit_time,0.0909201,0.0432583,0.0447857,0.0380628,0.0237623
mean_score_time,1.8681,1.62722,1.79505,1.18161,0.995706
std_score_time,0.0147114,0.0133231,0.0107439,0.0253525,0.0170416
param_randomforestclassifier__criterion,entropy,gini,gini,entropy,entropy
param_randomforestclassifier__n_estimators,140,117,131,74,59
param_simpleimputer__strategy,mean,median,mean,median,median
params,{'randomforestclassifier__criterion': 'entropy...,"{'randomforestclassifier__criterion': 'gini', ...","{'randomforestclassifier__criterion': 'gini', ...",{'randomforestclassifier__criterion': 'entropy...,{'randomforestclassifier__criterion': 'entropy...
split0_test_score,0.794209,0.792151,0.793161,0.793423,0.792675
split1_test_score,0.795129,0.795353,0.794119,0.793819,0.794194


In [0]:
# Make Prediction for test set

In [0]:
pipeline = search.best_estimator_

In [131]:
pipeline.fit(X_train, y_train)
print('Validation Accuracy', pipeline.score(X_test, y_test))

Validation Accuracy 0.812962962962963


In [0]:
# objective 2: submit predictions to kaggle competition 